In [ ]:
import requests, re, jmespath, csv

In [ ]:
# find api key in google drive
API_KEY = ""

baseURL = "https://api.scripture.api.bible/v1"
headers = {"api-key": API_KEY}

In [ ]:
def pipeline(bibleID, bibleName):

    # the most verses you can get at once is all verses of a chapter
    # so here we get the chapter IDs for each chapter in each book (excluding intros)
    # we use these IDs to request the verses for each chapter
    try:
        url = f"{baseURL}/bibles/{bibleID}/books"
        response = requests.get(url, headers=headers, params={"include-chapters": True})
        response.raise_for_status()  # exception if status not 200
        chapIDs = jmespath.search("data[].chapters[?number!='intro'].id[]", response.json())
    except requests.exceptions.RequestException as e:
        print(f"something went wrong\n{e}")
        return

    # make a csv
    with open(f"{bibleName}.csv", "w") as f:
        writer = csv.DictWriter(f, fieldnames=["citation", "verse"])
        writer.writeheader()

        # get verses
        for chapID in chapIDs:
            try:
                # request chapter info (includes verse text)
                url = f"{baseURL}/bibles/{bibleID}/chapters/{chapID}"
                response = requests.get(url, headers=headers, params={"content-type": "text"})
                response.raise_for_status()  # exception if status not 200
            except requests.exceptions.RequestException as e:
                print(f"something went wrong\n{e}")
                return

            # str of all verses in the chapter
            response = response.json()
            verses = response["data"]["content"]

            # split into individual verses
            verseList = re.split(r"\s*\[\d+\]\s*", verses.strip())
            verseList = [v.strip() for v in verseList if v]

            # write a line to the csv for each verse
            for i, verse in enumerate(verseList):
                writer.writerow({"citation": f"{response['data']['id']}.{i+1}", "verse": verse})
                print(f"wrote: {verse}")

### make csv for all the bibles

In [ ]:
bibleIDs = {
    "ASV": "06125adad2d5898a-01",
    "WEB": "72f4e6dc683324df-01",
    "FBV": "65eec8e0b60e656b-01",
    "WMB": "f72b840c855f362c-04",
    "KJV": "de4e12af7f28f599-02",
}


## uncomment to run

asv = pipeline(bibleIDs["ASV"], "asv")
# web = pipeline(bibleIDs["WEB"], "web")
# fbv = pipeline(bibleIDs["FBV"], "fbv")
# wmb = pipeline(bibleIDs["WMB"], "wmb")
# kjv = pipeline(bibleIDs["KJV"], "kjv")